In [3]:
import numpy as np
from tenpy.networks.mps import MPS
from tenpy.models.hubbard import FermiHubbardModel
from tenpy.algorithms import dmrg

import pickle
import matplotlib.pyplot as plt

In [17]:
from tenpy.networks.mps import MPS
from tenpy.algorithms import dmrg
from tenpy.models.hubbard import FermiHubbardModel

def run_dmrg_half_filling_2d(Lx, Ly, U, t, mu, chi_max, sweeps, hc_flag=False):
    model_params = {
        'Lx': Lx,
        'Ly': Ly,
        'U': U,
        't': t,
        'mu': mu,
        'bc_MPS': 'finite',  # Initial boundary conditions for finite DMRG
        'bc_x': 'open',      # Open boundaries in x-direction
        'bc_y': 'periodic',  # Periodic boundaries in y-direction
        'explicit_plus_hc': hc_flag
    }

    model = FermiHubbardModel(model_params)

    # Initialize the MPS for half-filling with an alternating pattern of up and down spins
    initial_state = ['up', 'down'] * (Lx * Ly // 2) +['up']
    psi = MPS.from_product_state(model.lat.mps_sites(), initial_state, bc='finite')

    dmrg_params = {
        'mixer': True,  # Enables mixing for better convergence in difficult phases
        'trunc_params': {
            'chi_max': chi_max,
            'svd_min': 1.e-10
        },
        'max_sweeps': sweeps,
    }

    # Growing the system in x-direction with iDMRG
    for _ in range(Lx):
        eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
        E0, psi = eng.run()

    # Switching to finite DMRG for the full system with PBC in y-direction
    model_params['bc_MPS'] = 'finite'
    model = FermiHubbardModel(model_params)
    eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
    E0, psi = eng.run()

    return E0, psi

# Example usage
Lx = 10  # Length in x-direction
Ly = 4   # Length in y-direction
U = 1.0
t = 1.0
mu = 0.0
chi_max = 100
sweeps = 10

E0, psi = run_dmrg_half_filling_2d(Lx, Ly, U, t, mu, chi_max, sweeps)
print("Ground state energy:", E0)


/home/gaurav/Projects/default/lib/python3.12/site-packages/tenpy/tools/params.py:231: UserWarning: unused options for config FermiHubbardModel:
['Lx', 'Ly', 'bc_y']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


ValueError: Length of p_state does not match number of sites.